In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import Vector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from azure.core.credentials import AzureKeyCredential
from langchain.prompts import PromptTemplate

openai_api_base = "https://xxxxx.openai.azure.com/"
openai_api_key = "xxxxxxxxxxxxxxxxx"
acs_base = "https://xxxx.search.windows.net/"
acs_key = "xxxxxxxxxxxxxxx"

embeddings = OpenAIEmbeddings(
    model = "text-embedding-ada-002",
    deployment = "xxxxxx",
    openai_api_type = "azure",
    openai_api_version = "2023-05-15",
    openai_api_base = openai_api_base,
    openai_api_key = openai_api_key
)

llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    deployment_name = "xxxxxxx",
    openai_api_type = "azure",
    openai_api_version = "2023-05-15",
    openai_api_base = openai_api_base,
    openai_api_key = openai_api_key
)

question = "Warum öffnet der Greifer nicht ganz?"

prompt = PromptTemplate(
    input_variables=["original_question", "search_results"],
    template="""
    Frage: {original_question}

    Verwende die folgenden Informationen aus diesen früheren Fehlerfällen:
    {search_results}
    """,
)

question_embedded=embeddings.embed_query(question)

search_client = SearchClient(
    acs_base,
    "issues",
    AzureKeyCredential(acs_key)
)
vector = Vector(
    value=question_embedded,
    k=5,
    fields="content_vector"
)

results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vectors=[vector],
    select=["content"],
))

contents = [result['content'] for result in results]

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
response = chain.run({"original_question": question, "search_results": contents})
print(response)